In [1]:
import os, csv, time
os.getcwd()

'/Users/Tristan/books/src'

In [2]:
# local modules
import config
import sparql

First we create an empty CSV file with the colums to fill.

## Find additional info

In [3]:
import config, sparql
os.listdir(config.dataset_dir)

['.DS_Store', 'test', 'output']

In [4]:
import pandas as pd
import sparql
filename = config.dataset_dir + '/output/bookdata_alt.csv'
print(filename)
info = pd.read_csv(filename)
info.keys()

../datasets//output/bookdata_alt.csv


Index(['filename', 'title', 'author', 'release year', 'genre', 'pos score',
       'neg score', 'neu score', 'comp score'],
      dtype='object')

In [5]:
info

,filename,title,author,release year,genre,pos score,neg score,neu score,comp score
0,289.txt,The Wind in the Willows,Kenneth Grahame,"July, 1995",NaN,NaN,NaN,NaN,NaN
1,504.txt,"The Fifth String, The Conspirators",John Philip Sousa,"April, 1996",NaN,NaN,NaN,NaN,NaN
2,262.txt,The Collected Poems of Rupert Brooke,Rupert Brooke,"May, 1995",NaN,NaN,NaN,NaN,NaN
3,510.txt,The Purcell Papers,Joseph Sheridan Le Fanu,"May 24, 2008",NaN,NaN,NaN,NaN,NaN
4,538.txt,Jean of the Lazy A,B. M. Bower,"May, 1996",NaN,NaN,NaN,NaN,NaN
5,706.txt,The Amateur Cracksman,E. W. Hornung,"November, 1996",NaN,NaN,NaN,NaN,NaN
6,712.txt,Thomas Jefferson,Edward S. Ellis et. al.,"January 21, 2006",NaN,NaN,NaN,NaN,NaN
7,29.txt,"Census of Population and Housing, 1990",United States Bureau of the Census,"August, 1992",NaN,NaN,NaN,NaN,NaN
8,15.txt,Moby Dick,Herman Melville,"May, 1991",NaN,NaN,NaN,NaN,NaN
9,699.txt,A Child's History of England,Charles Dickens,"May 6, 2007",NaN,NaN,NaN,NaN,NaN


In [6]:
keys, _ = sparql.book_info()

In [7]:
def books_info(title_list=[], verbose=False):
    
    print('searching %i titles' % len(title_list))
    data_per_book = []
    unknown_titles = []
    # keys, _ = sparql.book_info()
    for title in title_list:
        # TODO try a couple of times
        # book info returns     
        #  ('ok', (keys, [info_dict]) ) | ('error', 'http_error')
        (success, result) = sparql.book_info(title)
        if verbose: print(success, len(result[1]))
        if success == 'ok':
            _, results = result
            if not results == []:
                # ignore all but the first result
                # TODO save results that have an author
                data_per_book.append(results[0])
            else: # 'http_error'
                if verbose: print('error', title)
                unknown_titles.append(title)
        
 
    return data_per_book, unknown_titles



In [8]:
v = False
results, unknown_titles = books_info(info['title'], verbose=v)

print('n results',len(results))
print('n failed (not found)',len(unknown_titles))



searching 698 titles
n results 285
n failed (not found) 411


In [11]:
def find_genres(dic):
    to_strip = 'http://dbpedia.org/resource/'
    genre_list = []
    title_list = []
    for result in dic:
        print(result)
        try:
            genre = (result['genre']['value'])
            genre = genre.replace(to_strip, '')
            genre = genre.replace('_', ' ')
            genre_list.append(genre)
            
            title = result['book']['value']
            title = title.replace(to_strip, '')
            title = title.replace('_', ' ')
            title_list.append(title)
            
        except KeyError as error:
            # Output expected KeyErrors.
            continue
            
#   COMMENTED CODE DOES NOT WORK YET    
#   genres = pd.Series(genre_list, title_list)
#   print(genres)
#   info['genre'] = genres.values
    unique_genres = set(genre_list)
    return genres, unique_genres

genres, unique_genres = find_genres(results)



{'book': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Wind_in_the_Willows'}, 'author': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Kenneth_Grahame'}, 'genre': {'type': 'uri', 'value': "http://dbpedia.org/resource/Children's_literature"}, 'birthDate': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1859-03-08'}}
{'book': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Purcell_Papers'}, 'author': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Joseph_Sheridan_Le_Fanu'}}
{'book': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Amateur_Cracksman'}, 'author': {'type': 'uri', 'value': 'http://dbpedia.org/resource/E.W._Hornung'}, 'genre': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Crime_fiction'}}
{'book': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Thomas_Jefferson:_Author_of_America'}, 'author': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Christopher_Hitchens'}, 'birthDat

ValueError: Length of values does not match length of index

In [ ]:
sanitized = [sparql.sanitize(t) for t in unknown_titles]
results, unknown_titles = books_info(sanitized, verbose=v)
print('n results',len(results))
print('n failed (not found)',len(unknown_titles))

In [ ]:
# TODO string.strip
string = "sdf ' df"
string = string.strip('"`')
string = string.strip("'")
string

In [ ]:
info['title'][0:10]